In [ ]:
#!pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [4]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [5]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [6]:
X=scaler.fit_transform(x)

In [7]:
X.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=1)

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Input,Dropout

In [10]:
model=Sequential()
#model.add(Input(shape=(8,)))
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4335 - loss: 0.7930 - val_accuracy: 0.5065 - val_loss: 0.7478
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5494 - loss: 0.7335 - val_accuracy: 0.6299 - val_loss: 0.6804
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6445 - loss: 0.6654 - val_accuracy: 0.6818 - val_loss: 0.6295
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7026 - loss: 0.6131 - val_accuracy: 0.7078 - val_loss: 0.5887
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7156 - loss: 0.5818 - val_accuracy: 0.7532 - val_loss: 0.5582
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7180 - loss: 0.5526 - val_accuracy: 0.7403 - val_loss: 0.5374
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7306 - loss: 0.5453 - val_accuracy: 0.7532 - val_loss: 0.5226
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7040 - loss: 0.5365 - val_accuracy: 0.7792 - 

In [13]:
import keras_tuner as kt

In [ ]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  #optimizer = tf.keras.optimizers.get(optimizer)
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='Optimizer')

In [ ]:
tuner.search(x_train,y_train,validation_data=(x_test,y_test),epochs=5)

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 12s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [ ]:
 model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7250 - loss: 0.5683 - val_accuracy: 0.7792 - val_loss: 0.5490
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7408 - loss: 0.5544 - val_accuracy: 0.7792 - val_loss: 0.5408
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7750 - loss: 0.5197 - val_accuracy: 0.7792 - val_loss: 0.5352
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7613 - loss: 0.5307 - val_accuracy: 0.7792 - val_loss: 0.5299
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7368 - loss: 0.5379 - val_accuracy: 0.7857 - val_loss: 0.5251
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7488 - loss: 0.5232 - val_accuracy: 0.7922 - val_loss: 0.5213
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7717 - loss: 0.5122 - val_accuracy: 0.7922 - val_loss: 0.5178
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7474 - loss: 0.5225 - val_accuracy: 0.79

In [ ]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('units',min_value=8,max_value=128)#,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='NumberNodes')

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 13s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 31}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(x_train,y_train,batch_size=32,initial_epoch=6,epochs=100,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7570 - loss: 0.5446 - val_accuracy: 0.7922 - val_loss: 0.5135
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7354 - loss: 0.5227 - val_accuracy: 0.7857 - val_loss: 0.4964
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7320 - loss: 0.5258 - val_accuracy: 0.7857 - val_loss: 0.4834
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7494 - loss: 0.5027 - val_accuracy: 0.7857 - val_loss: 0.4756
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7509 - loss: 0.5017 - val_accuracy: 0.7792 - val_loss: 0.4702
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7569 - loss: 0.4751 - val_accuracy: 0.7792 - val_loss: 0.4652
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7720 - loss: 0.4778 - val_accuracy: 0.7857 - val_loss: 0.4619
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7702 - loss: 0.4793 - val_accuracy: 0.78

In [ ]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(31,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(31,activation='relu',))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [ ]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='Numberoflayers')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 09s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(x_train,y_train,initial_epoch=6,epochs=100,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7275 - loss: 0.6643 - val_accuracy: 0.7403 - val_loss: 0.6529
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7103 - loss: 0.6576 - val_accuracy: 0.6883 - val_loss: 0.6434
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7202 - loss: 0.6440 - val_accuracy: 0.6818 - val_loss: 0.6351
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7187 - loss: 0.6268 - val_accuracy: 0.6688 - val_loss: 0.6266
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6746 - loss: 0.6241 - val_accuracy: 0.6623 - val_loss: 0.6185
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7016 - loss: 0.6081 - val_accuracy: 0.6623 - val_loss: 0.6114
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6768 - loss: 0.6051 - val_accuracy: 0.6623 - val_loss: 0.6042
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6606 - loss: 0.6042 - val_accuracy: 0.66

In [14]:
def build_model(hp):
  model=Sequential()
  counter=0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int('units'+str(i),min_value=1,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['sigmoid','tanh','relu']), input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i),min_value=1,max_value=128,step=8),activation=hp.Choice('activation'+str(i),values=['sigmoid','tanh','relu'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','sgd','adam','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
  return model




In [15]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='lay_unit_opt_drop')

Reloading Tuner from mydir/lay_unit_opt_drop/tuner0.json


In [16]:
tuner.search(x_train,y_train,validation_data=(x_test,y_test),epochs=5)

In [17]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 41,
 'activation0': 'tanh',
 'dropout0': 0.9,
 'optimizer': 'sgd',
 'units1': 17,
 'activation1': 'sigmoid',
 'dropout1': 0.7,
 'units2': 89,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 33,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 41,
 'activation4': 'tanh',
 'dropout4': 0.4,
 'units5': 105,
 'activation5': 'tanh',
 'dropout5': 0.7,
 'units6': 89,
 'activation6': 'tanh',
 'dropout6': 0.5,
 'units7': 81,
 'activation7': 'sigmoid',
 'dropout7': 0.2}

In [18]:
model=tuner.get_best_models(num_models=1)[0]

In [19]:
model.fit(x_train,y_train,epochs=200,initial_epoch=5,validation_data=(x_test,y_test))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6312 - loss: 0.6742 - val_accuracy: 0.6429 - val_loss: 0.6488
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6199 - loss: 0.6832 - val_accuracy: 0.6429 - val_loss: 0.6487
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6188 - loss: 0.6763 - val_accuracy: 0.6429 - val_loss: 0.6477
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6214 - loss: 0.6685 - val_accuracy: 0.6429 - val_loss: 0.6454
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6186 - loss: 0.6783 - val_accuracy: 0.6429 - val_loss: 0.6433
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6260 - loss: 0.6938 - val_accuracy: 0.6429 - val_loss: 0.6445
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6561 - loss: 0.6561 - val_accuracy: 0.6429 - val_loss: 0.6452
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6564 - loss: 0.6411 - val_accuracy: 0.642